In [16]:
import os
import sys 
import numpy as np 
import tensorflow as tf 
import time 
batch_size =50 

def load_model(session, save_path):
    """ Loads a saved TF model from a file.Returns:The inputs placehoder """
    print("Loading model from file '%s'..." % (save_path))
    meta_file = save_path + ".meta"
    if not os.path.exists(meta_file):
        print("ERROR: Expected .meta file '%s', but could not find it." % \
        (meta_file))
        sys.exit(1) 
    saver = tf.train.import_meta_graph(meta_file)
    save_path = os.path.join("./", save_path) 
    saver.restore(session, save_path)  
    return extract_validation_handles(session)
    
def load_validation_data( model,filename):
    '''load the testing dataset with the process and the name of testing file''' 
    global batch_size 
    """ Loads the validation data,  Returns: A tuple of the loaded validation data and validation labels. """
    print("Loading validation data...")
    folder = 'sub_features/'+ filename  
    if model ==1: # SP process
        testX =np.load('E:/identification_data/01_PSSE_datasets/Dataset/saved_data_jupyter/'+folder+'/testX_pred.npy' ) 
        testY =np.load('E:/identification_data/01_PSSE_datasets/Dataset/saved_data_jupyter/'+folder+'/testY_pred.npy'  ) 
        Energy =np.load('E:/identification_data/01_PSSE_datasets/Dataset/saved_data_jupyter/'+folder+'/Energy_pred.npy' )
    elif model==2: # NS process
        testX =np.load('E:/identification_data/01_PSSE_datasets/Dataset/saved_data_jupyter/'+folder+'/testX_no.npy' ) 
        testY =np.load('E:/identification_data/01_PSSE_datasets/Dataset/saved_data_jupyter/'+folder+'/testY_no.npy' ) 
        Energy =np.load('E:/identification_data/01_PSSE_datasets/Dataset/saved_data_jupyter/'+folder+'/Energy_no.npy'  ) 
    batch_size =np.shape(testX)[0]  
    print (batch_size)
    return (testX,testY,Energy)  

def validate_model(session, val_data, x,y, e, phase_train,prob ): 
    """ Validates the model stored in a session.Returns:The overall validation accuracy for the model. """
    global batch_size
    print("Validating model...") 
    predict_op = tf.argmax(prob, 1)
    correct = tf.equal(predict_op,tf.argmax(y, 1))
    acc= tf.reduce_mean(tf.cast(correct, tf.float32))  
    val_x, val_y, val_e = val_data
    test_correct = session.run(correct,feed_dict={x: val_x, e:val_e,y :val_y , phase_train : True  })
    test_acc = session.run(acc,feed_dict={x: val_x, e:val_e,y :val_y , phase_train : True  })
    return ( test_acc), test_correct  

def extract_validation_handles(session):
    """ Extracts the input and predict_op handles that we use for validation.
    Args:
      session: The session with the loaded graph.
    Returns:
      The inputs placeholder  """ 
    valid_nodes = tf.get_collection_ref("validation_nodes") 
    x = valid_nodes[0] 
    y = valid_nodes[1]
    e = valid_nodes[2]  
    phase_train =valid_nodes[3]
    prob = valid_nodes[4]
    global batch_size  
    return (x,y,e, phase_train,prob)

def each_perform(correct_results,eval_labels ):
    label_y=np.argmax(eval_labels,1) 
    
    acc = []
    zero = np.where(label_y==0) 
    correctl=[correct_results[i] for i in zero]
    accuracy_zero= np.mean(correctl,dtype=np.float32)
    acc.append(accuracy_zero)

    one = np.where(label_y==1)
    correctg=[correct_results[i] for i in one] 
    accuracy_one= np.mean(correctg,dtype=np.float32)
    acc.append(accuracy_one)

    two = np.where(label_y==2)
    correctt=[correct_results[i] for i in two] 
    accuracy_two= np.mean(correctt,dtype=np.float32)
    acc.append(accuracy_two) 
    return acc, correctl,correctg,correctt


def reset():
    tf.reset_default_graph() 
model = 2# model = 1: SP process; model = 2: NS process
folder ='fully_best/modelA_ratio100_set01'#'ratio_80_best/modelA_ratio80_set4'####'modelA_fully3'##''modelA_ratio100_set3'#'modelA_ratio80_set4'#'modelA_ratio50_set2'#'modelA_ratio30_set2'#'modelA_fully3'#'modelA_ratio24_set05'#'modelA_small04' # 'modelA_fully3' #,'save2','save3','save4','save5','save6','save7','save8','save9','save10','save11','save12','save13','save14','save15' ]
testname = 'one_sec'
if testname == 'one_sec':
    used = [768, 4, 261, 262, 11, 12, 791, 26, 177, 951, 957, 192, 964, 716, 717, 980, 985, 219, 733, 227, 229, 741, 1127, 250, 1135, 752, 753, 1139, 1140, 756, 757, 246, 1146, 1149]
elif testname == 'two_sec':
    used = [385, 646, 392, 650, 656, 400, 401, 660, 407, 410, 155, 420, 423, 427, 450, 581, 1225, 591, 1237, 1238, 342, 991, 1248, 1249, 352, 1004, 627, 632, 381]
elif testname == 'half_sec':
    used = [282, 288, 1058, 1063, 1070, 1198, 690, 1203, 1076, 308, 1079, 1208, 315, 700, 64, 708, 709, 1094, 1222, 1098, 76, 79, 1107, 86, 1114, 866, 879, 113, 888, 890]
elif testname == 'one_half_sec':
    used = [1024, 1159, 1162, 1044, 534, 1046, 536, 1175, 540, 1054, 159, 928, 1185, 548, 549, 933, 40, 941, 46, 561, 52, 821, 949, 950, 572, 834, 463, 854, 479, 481, 501, 510, 1023]
with tf.Session() as session:  
    best_model = 'E:/05_research/04_deep_learning_model/Save_parameters/modelA/'+folder# os.path.join(save_path,model_name)
    x,y,e,phase_train,prob = load_model(session, best_model)
    val_data = load_validation_data(model, testname) # the predicted mode
    print (np.shape(val_data[0])[0])
    total = [i for i in range(np.shape(val_data[0])[0])]
    remain = list(set(total) - set(used)); 
    accuracy ,correct_results= validate_model(session, val_data, x,y,e,phase_train,prob) 
    remain_results = np.array([correct_results[i] for i in remain] )
    labels = [val_data[1][i] for i in remain]
    #print ("Overall validation accuracy is %f \n" %(accuracy))
    acc,correct_l,correct_g,correct_t= each_perform(remain_results,labels ) 
    #reset() 
    session.close()     
id_True =  np.nonzero(remain_results==True) 
overall_acc = (len(id_True[0])/len(remain_results))

print ('LT:', 100*np.mean( correct_l))
print ('GT:', 100*np.mean( correct_g))
print ('TP:', 100*np.mean( correct_t))
print ('Overall:', 100*overall_acc) 
    

Loading model from file 'E:/05_research/04_deep_learning_model/Save_parameters/modelA/fully_best/modelA_ratio100_set01'...
INFO:tensorflow:Restoring parameters from E:/05_research/04_deep_learning_model/Save_parameters/modelA/fully_best/modelA_ratio100_set01
Loading validation data...
322
322
Validating model...
LT: 79.38144329896907
GT: 68.31683168316832
TP: 96.3963963963964
Overall: 81.87702265372168


In [52]:
id_f  =[ 15,   20,   22,   27,   29,   34,   61,   62,   66,   68,   69,
          72,   73,   75,   77,   79,   90,   93,  106,  131,  143,  148,
         184,  225,  257,  262,  345,  387,  447,  499,  547,  565,  566,
         569,  578,  582,  584,  585,  586,  588,  590,  626,  648,  667,
         669,  674,  675,  676,  714,  739,  740,  741,  742,  747,  749,
         751,  752,  754,  755,  756,  760,  762,  763,  764,  765,  766,
         767,  769,  772,  773,  776,  777,  778,  780,  783,  792,  796,
         803,  805,  824,  825,  833,  839,  843,  844,  848,  849,  850,
         851,  866,  886,  896,  906,  907,  929,  932,  948,  954,  962,
         987, 1008, 1026, 1039, 1040, 1042, 1060, 1063, 1064, 1065, 1066,
        1068, 1071, 1078, 1079, 1080, 1098, 1103, 1108, 1110, 1124, 1125,
        1126, 1127, 1130, 1131, 1132, 1133, 1134]